In [2]:
import pandas as pd
import my_predictor as mp
import gdi_loader as gdi
from sklearn import svm
import bert_swiss_lm as bsl
import itertools
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [2]:
my_predictor = mp.MyPredictor("train_embedding_bert_swiss_lm.feather", "dev_embedding_bert_swiss_lm.feather", None)

In [3]:
my_predictor.fit()

In [4]:
my_predictor.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.91      0.93      0.92      3750
          BS       0.90      0.88      0.89      3269
          LU       0.92      0.85      0.88      3390
          ZH       0.89      0.95      0.92      3870

    accuracy                           0.90     14279
   macro avg       0.91      0.90      0.90     14279
weighted avg       0.91      0.90      0.90     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.63      0.62      0.62      1053
          BS       0.71      0.59      0.65      1528
          LU       0.61      0.47      0.53      1017
          ZH       0.56      0.86      0.68       932

    accuracy                           0.63      4530
   macro avg       0.63      0.64      0.62      4530
weighted avg       0.64      0.63      0.62      4530



In [3]:
my_predictor_pooler = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                     ["sentence_embedding_first_state", "audio"],
                                     audio_length=10)

my_predictor_pooler.fit()
my_predictor_pooler.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.78      0.83      0.81      3750
          BS       0.74      0.73      0.73      3269
          LU       0.74      0.67      0.71      3390
          ZH       0.80      0.82      0.81      3870

    accuracy                           0.77     14279
   macro avg       0.76      0.76      0.76     14279
weighted avg       0.77      0.77      0.77     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.57      0.60      0.58      1053
          BS       0.68      0.54      0.60      1528
          LU       0.53      0.45      0.49      1017
          ZH       0.57      0.81      0.67       932

    accuracy                           0.59      4530
   macro avg       0.59      0.60      0.58      4530
weighted avg       0.60      0.59      0.59      4530



In [8]:
my_predictor_with_audio = mp.MyPredictor("train_embedding_bert_swiss_lm.feather", "dev_embedding_bert_swiss_lm.feather", None, "sentence_embedding_pooler")
my_predictor_with_audio.fit()
my_predictor_with_audio.predict_all()

KeyError: 's'

In [3]:
my_predictor_pooler.fit()

In [3]:
for i in (0.0001, 0.00006, 0.00003):
    # best C: 0.00006 (macro f1 0.33)
    # standard_Scaler= True: 0.0006 (accuracy 0.31) schlechter als 0.00006 (accuracy 0.38)
    # standard_Scaler= False: 0.0006 (accuracy )  als normalize_each_vector=False 0.00006 (accuracy 0.31)
#    for standard_scalar in (False, True):
    for kernel in ("linear", "rbf"):
        standard_scalar = True
        norm = False
        my_predictor_audio = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                     "audio",
                                    normalize_each_vector=norm)

        my_predictor_audio.fit(C=i, standard_scaler=standard_scalar, kernel=kernel)
        print("C: ", i, "kernel: ", kernel)
        my_predictor_audio.predict_all()

KeyboardInterrupt: 

In [5]:
for standard_scalar in (False,):
    for norm in (False,):
        my_predictor_audio_gaussian = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                                     ("sentence_embedding_first_state", "audio"),
                                    normalize_each_vector=norm)

        my_predictor_audio_gaussian.fit_gaussian_naive_bayes(standard_scaler=standard_scalar)
        print("standard_scaler: ", standard_scalar, "normalize_each_vector: ", norm)
        my_predictor_audio_gaussian.predict_all()

standard_scaler:  False normalize_each_vector:  False
Training set:
              precision    recall  f1-score   support

          BE       0.85      0.84      0.84      3750
          BS       0.80      0.82      0.81      3269
          LU       0.82      0.75      0.78      3390
          ZH       0.81      0.87      0.84      3870

    accuracy                           0.82     14279
   macro avg       0.82      0.82      0.82     14279
weighted avg       0.82      0.82      0.82     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.66      0.59      0.62      1053
          BS       0.80      0.77      0.79      1528
          LU       0.81      0.57      0.67      1017
          ZH       0.60      0.90      0.72       932

    accuracy                           0.71      4530
   macro avg       0.72      0.71      0.70      4530
weighted avg       0.73      0.71      0.71      4530



In [13]:
print(precision_score(lables_dev, prediction_dev, average="macro"))
print(classification_report(lables_dev, prediction_dev))

0.6147505478251539
              precision    recall  f1-score   support

          BE       0.60      0.62      0.61       242
          BS       0.69      0.52      0.60       335
          LU       0.69      0.49      0.57       235
          ZH       0.48      0.84      0.61       188

    accuracy                           0.60      1000
   macro avg       0.61      0.62      0.60      1000
weighted avg       0.63      0.60      0.60      1000

